this script takes individual accessibility files that have been generated on euler and merges them into a single dataset + calculates differences for every person

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import geopandas
import r5py
import shapely
import pandas as pd
import geopandas as gpd
import numpy as np
import datetime
import copy

import os
import snman
from snman import osmnx_customized as oxc
from snman.constants import *

PERIMETER = '_accessibility_debug'

# Set these paths according to your own setup
data_directory = os.path.join('C:',os.sep,'Users','lballo','polybox','Research','SNMan','SNMan Shared','data_v2')
inputs_path = os.path.join(data_directory, 'inputs')
process_path = os.path.join(data_directory, 'process', PERIMETER)
outputs_path = os.path.join(data_directory, 'outputs', PERIMETER)

CRS_internal = 2056      # for Zurich

In [ ]:
euler_files_path = os.path.join(outputs_path, 'euler')

In [64]:
def add_pseudo_person_id(accessibility):
    """
    add surrogate person id 
    """
    accessibility['record'] = accessibility.apply(
        lambda row: f"{row['statent_id']}_{row['age']}_{row['sex']}_{row['maritalstatus']}_{row['residencepermit']}_{row['geometry']}",
        axis=1
    )
    accessibility['record2'] = accessibility.groupby('record').cumcount()
    accessibility['record'] = accessibility['record'] + accessibility['record2'].astype(str)

def merge_for_state(state):
    """
    Merge all partial datasets into one
    """

    files = os.listdir(euler_files_path)
    files = list(
        filter(
            lambda item: (item[-5:] == '.gpkg') & (state in item),
            files
        )
    )
    datasets = map(
        lambda file: snman.io.import_geofile_to_gdf(
            str(os.path.join(euler_files_path, file))
        ).reset_index(),
        files
    )
    accessibility = gpd.GeoDataFrame(
        pd.concat(datasets)
    )
    accessibility['state'] = state
    add_pseudo_person_id(accessibility)
    return accessibility
    
before = merge_for_state('before')
after = merge_for_state('after')

In [83]:
# merge before and after and calculate differences

data_fields = ['accessibility', 'accessibility_private_cars', 'accessibility_transit', 'accessibility_cycling', 'accessibility_foot']
m = pd.merge(before, after[data_fields + ['record']], on='record', suffixes=['_before', '_after'])

for field in data_fields:
    m[f'{field}_diff'] = m[f'{field}_after'] - m[f'{field}_before']
    m[f'{field}_diff_perc'] = (m[f'{field}_after'] - m[f'{field}_before']) / m[f'{field}_before']

snman.io.export_gdf(
    m,
    os.path.join(outputs_path, 'accessibility_diff.gpkg')
) 

m

,index,age,sex,maritalstatus,residencepermit,residentpermit,statent_id,accessibility_before,accessibility_cycling_before,accessibility_foot_before,...,accessibility_diff,accessibility_diff_perc,accessibility_private_cars_diff,accessibility_private_cars_diff_perc,accessibility_transit_diff,accessibility_transit_diff_perc,accessibility_cycling_diff,accessibility_cycling_diff_perc,accessibility_foot_diff,accessibility_foot_diff_perc
0,0,38,1,1,301,3,68672447.0,449.042529,574.198628,227.824450,...,-3.029201,-0.006746,-162.928731,-0.271954,0.0,0.0,72.349107,0.126000,-0.438864,-0.001926
1,1,48,2,2,-2,-2,68672447.0,449.042045,574.198628,227.824450,...,-3.029498,-0.006747,-162.928731,-0.271954,0.0,0.0,72.349107,0.126000,-0.438864,-0.001926
2,2,56,1,2,201,2,68672447.0,449.041551,574.198628,227.824450,...,-3.029646,-0.006747,-162.928731,-0.271954,0.0,0.0,72.349107,0.126000,-0.438864,-0.001926
3,3,13,1,1,-2,-2,69702513.0,331.593569,404.511378,138.711689,...,13.525293,0.040789,27.871188,0.055322,0.0,0.0,11.134704,0.027526,0.079790,0.000575
4,4,20,2,1,-2,-2,69702513.0,331.593459,404.511378,138.711689,...,13.525322,0.040789,27.871188,0.055322,0.0,0.0,11.134704,0.027526,0.079790,0.000575
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40690,6601,30,2,1,-2,-2,68222474.0,506.383599,687.447518,296.087981,...,-3.183156,-0.006286,-186.584144,-0.269320,0.0,0.0,49.234949,0.071620,0.852137,0.002878
40691,6602,34,2,2,302,3,68222474.0,506.383551,687.447518,296.087981,...,-3.183539,-0.006287,-186.584144,-0.269320,0.0,0.0,49.234949,0.071620,0.852137,0.002878
40692,6603,0,2,1,-2,-2,68222474.0,506.383773,687.447518,296.087981,...,-3.181736,-0.006283,-186.584144,-0.269320,0.0,0.0,49.234949,0.071620,0.852137,0.002878
40693,6604,49,1,2,202,2,68222474.0,506.383387,687.447518,296.087981,...,-3.184716,-0.006289,-186.584144,-0.269320,0.0,0.0,49.234949,0.071620,0.852137,0.002878
